In [ ]:
!pip install gradio

In [ ]:
import os
import json
import gradio as gr
from openai import OpenAI

client = OpenAI(
    api_key="sk-3cc94a27441a472395f00ba59dd1634f",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

def format_history(chat_history):
    """转换历史记录，过滤思考内容"""
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    for msg in chat_history:
        messages.append({"role": "user", "content": msg[0]})
        if msg[1]:
            # 提取最后回复内容（过滤思考内容）
            final_reply = msg[1].split("最终回复：")[-1].strip()
            messages.append({"role": "assistant", "content": final_reply})
    return messages

def predict(message, chat_history):
    """处理包含思考内容的流式响应"""
    chat_history.append((message, ""))
    messages = format_history(chat_history[:-1])
    messages.append({"role": "user", "content": message})

    completion = client.chat.completions.create(
        model="qwen3-235b-a22b",
        messages=messages,
        stream=True,
        stream_options={"include_usage": True},
        extra_body={"enable_thinking": True},
    )

    full_response = ""
    full_reasoning = ""
    current_reasoning = ""
    for chunk in completion:
        chunk_data = chunk.model_dump()
        if not chunk_data.get('choices') or len(chunk_data['choices']) == 0:
          continue  # 跳过无效的chunk
        delta = chunk_data['choices'][0]['delta']

        # 解析思考内容
        if delta.get('reasoning_content'):
            current_reasoning = delta['reasoning_content']
            full_reasoning += current_reasoning

        # 解析正式回复内容
        if delta.get('content'):
            full_response += delta['content']

        # 构建显示内容
        display_text = ""
        if full_reasoning != '':
            display_text = "🧠 思考过程：\n" + full_reasoning
        if full_response:
            display_text += f"\n💡 最终回复：\n{full_response}"

        # 更新聊天记录
        if current_reasoning or delta.get('content'):
            chat_history[-1] = (message, display_text)
            yield chat_history

        # 重置当前思考内容（避免重复显示）
        if delta.get('content'):
            current_reasoning = ""

# 自定义样式
css = """
.gr-chatbot {min-height: 600px;}
.gr-chatbot .assistant-message {white-space: pre-wrap;}
.dark .reasoning {color: #90CAF9;}
.reasoning {color: #1E88E5; font-style: italic;}
"""

with gr.Blocks(css=css) as demo:
    gr.Markdown("# <center>🤖 千问大模型智能对话（含思考过程）</center>")

    with gr.Row():
        chatbot = gr.Chatbot(
            bubble_full_width=False,
            render_markdown=True,
        )

    with gr.Row():
        msg = gr.Textbox(
            placeholder="请输入您的问题...",
            container=False,
            scale=5
        )
        clear = gr.Button("🗑️ 清空对话", scale=1)

    msg.submit(predict, [msg, chatbot], chatbot)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0",debug=False)